# S&P 500 Index Fund Weighted Equally

## Introduction

The S&P 500 is an incredibly popular stock market index. This index weighs all its stocks in proportion to their market capitalization. Additionally, it contains 500 major companies, but slightly over 500 stocks as some companies have multiple listed stocks based on class.

This project aims to recalculate the S&P500 so that every stock is weighted equally. The goal of this section is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd
import requests
import xlsxwriter
import math

## List of S&P500 Stocks to be imported

Next, the constituents of the S&P 500 need to be imported.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

This project uses a static version of the S&P 500 constituents.

Now it's time to import these stocks into the Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Next, I will import the IEX Cloud API token. This is the data provider that will be used throughout this project.

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

## Making an initial API Call

Now it's time to structure API calls to IEX cloud. 

Ideally, the following information is needed from the API endpoint:

* Market capitalization for each stock
* Price of each stock



In [ ]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing the API Call

The API call that was executed in the last code block contains all the information required to build our equal-weight S&P 500 strategy. 

Now, the data needs to be parsed and formatted appropriately.

In [ ]:
data['latestPrice']
data['marketCap']

## Adding Stocks Data to a Pd DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [ ]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

In [ ]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

## Looping Through All Tickers in the List of Stocks

Using the same logic utilized previously, data can be pulled for all S&P 500 stocks and stored in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)


## Using Batch API Calls to Improve Performance

Batch API calls will significantly improve the performance of the code because HTTP requests are a really slow component of the script.

IEX Cloud limits their batch API calls to 100 tickers per request. This reduces the number of API calls from 500 to 5!

In this section, the list of stocks are split into groups of 100 and then a batch API call is made for each group.

In [ ]:
"""From lst, yield successive n-sized chunks."""

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

## Calculating the Number of Shares to Buy

Now, very importantly, the number of shares of each stock to buy need to be calculated.

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

## Formatting the Excel Output

The XlsxWriter library for Python will help with creating well-formatted Excel files.

### Initializing the XlsxWriter Object

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating Formats needed for the `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. Four main formats are needed for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of the `.xlsx` File

The `set_column` method can be applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of spreadsheets.

In [ ]:
"""
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
"""


This is an example of code that works, but it is incredibly repetitive. 

A more efficient version:

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving the Final Excel Output

Saving the Excel file is very easy using xlsxwriter:

In [ ]:
writer.save()